# Imports

In [1]:
import joblib

import numpy   as np
import pandas  as pd

from lightgbm       import LGBMRegressor
from feature_engine import creation

from sklearn.metrics           import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection   import cross_validate
from sklearn.feature_selection import SelectFromModel

## Functions

In [2]:
random_state = 42
np.random.seed(random_state)

In [3]:
def jupyter_settings():
    
    pd.options.display.max_columns = None
    pd.options.display.max_rows = 500

jupyter_settings()

In [4]:
def get_cross_validate_metrics(dict_cv_result):

    print(f"""
    R2 test mean: {dict_cv_result['test_r2'].mean()}; 
    R2 test std: {dict_cv_result['test_r2'].std()}

    MAE test mean: {dict_cv_result['test_mae'].mean()}; 
    MAE test std: {dict_cv_result['test_mae'].std()}
    
    RMSE test mean: {dict_cv_result['test_rmse'].mean()}; 
    RMSE test std: {dict_cv_result['test_rmse'].std()}
    """)

In [5]:
def get_metrics(y_true, y_pred):

    r2 = r2_score(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)

    print(f"R2: {r2}; RMSE: {rmse}; MAE: {mae}; MAPE: {mape}")

# Loading Datasets

In [6]:
df_train = pd.read_parquet('../data/processed/v2_dataframe_train.parquet')
df_oot = pd.read_parquet('../data/processed/v2_dataframe_oot.parquet')

In [7]:
df_train.head()

,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng,seller_geolocation_city_count,seller_geolocation_city_nunique,seller_geolocation_city_size,seller_geolocation_city_mode,seller_geolocation_state_count,seller_geolocation_state_nunique,seller_geolocation_state_size,seller_geolocation_state_mode,distance_customer_seller_count,distance_customer_seller_min,distance_customer_seller_max,distance_customer_seller_mean,distance_customer_seller_median,distance_customer_seller_q25,distance_customer_seller_q75,distance_customer_seller_range,payment_sequential_count,payment_sequential_min,payment_sequential_max,payment_sequential_mean,payment_sequential_median,payment_sequential_q25,payment_sequential_q75,payment_sequential_range,payment_type_count,payment_type_nunique,payment_type_size,payment_type_mode,payment_installments_count,payment_installments_min,payment_installments_max,payment_installments_mean,payment_installments_median,payment_installments_q25,payment_installments_q75,payment_installments_range,payment_value_count,payment_value_min,payment_value_max,payment_value_mean,payment_value_median,payment_value_q25,payment_value_q75,payment_value_range,product_id_count,seller_id_count,price_count,price_min,price_max,price_mean,price_median,price_q25,price_q75,price_range,freight_value_count,freight_value_min,freight_value_max,freight_value_mean,freight_value_median,freight_value_q25,freight_value_q75,freight_value_range,product_category_name_count,product_category_name_nunique,product_category_name_size,product_category_name_mode,product_name_lenght_count,product_name_lenght_min,product_name_lenght_max,product_name_lenght_mean,product_name_lenght_median,product_name_lenght_q25,product_name_lenght_q75,product_name_lenght_range,product_description_lenght_count,product_description_lenght_min,product_description_lenght_max,product_description_lenght_mean,product_description_lenght_median,product_description_lenght_q25,product_description_lenght_q75,product_description_lenght_range,product_photos_qty_count,product_photos_qty_min,product_photos_qty_max,product_photos_qty_mean,product_photos_qty_median,product_photos_qty_q25,product_photos_qty_q75,product_photos_qty_range,product_weight_g_count,product_weight_g_min,product_weight_g_max,product_weight_g_mean,product_weight_g_median,product_weight_g_q25,product_weight_g_q75,product_weight_g_range,product_length_cm_count,product_length_cm_min,product_length_cm_max,product_length_cm_mean,product_length_cm_median,product_length_cm_q25,product_length_cm_q75,product_length_cm_range,product_height_cm_count,product_height_cm_min,product_height_cm_max,product_height_cm_mean,product_height_cm_median,product_height_cm_q25,product_height_cm_q75,product_height_cm_range,product_width_cm_count,product_width_cm_min,product_width_cm_max,product_width_cm_mean,product_width_cm_median,product_width_cm_q25,product_width_cm_q75,product_width_cm_range,order_purchase_until_approved_in_seconds,order_purchase_until_approved_in_minutes,order_purchase_until_approved_in_hours,order_purchase_until_approved_in_days,order_purchase_timestamp_second,order_purchase_timestamp_minute,order_purchase_timestamp_hour,order_purchase_timestamp_day,order_purchase_timestamp_month,order_approved_at_second,order_approved_at_minute,order_approved_at_hour,order_approved_at_day,order_approved_at_month,delivered_in_days,estimated_delivery_in_days,time
0,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18,3149,sao paulo,SP,-23.574809,-46.587471,1,1,1,maua,1,1,1,SP,1,18.051106,18.051106,18.051106,18.051106,18.051106,18.051106,0.0,3.0,1.0,3.0,2.0,2.0,1.5,2.5,2.0,3.0,2.0,3.0,voucher,3.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,3.0,2.00,18.59,12.903333,18.12,10.06,18.355,16.59,1.0,1.0,1.0,29.99,29.99,29.99,29.99,29.99,29.99,0.0,1.0,8.72,8.72,8.72,8.72,8.72,8.72,0.0,1.0,1.0,1.0,utilidades_domesticas,1.0,40.0,40.0,40.0,40.0,40.0,40.0,0.0,1.0,268.0,26

In [8]:
df_oot.head()

,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng,seller_geolocation_city_count,seller_geolocation_city_nunique,seller_geolocation_city_size,seller_geolocation_city_mode,seller_geolocation_state_count,seller_geolocation_state_nunique,seller_geolocation_state_size,seller_geolocation_state_mode,distance_customer_seller_count,distance_customer_seller_min,distance_customer_seller_max,distance_customer_seller_mean,distance_customer_seller_median,distance_customer_seller_q25,distance_customer_seller_q75,distance_customer_seller_range,payment_sequential_count,payment_sequential_min,payment_sequential_max,payment_sequential_mean,payment_sequential_median,payment_sequential_q25,payment_sequential_q75,payment_sequential_range,payment_type_count,payment_type_nunique,payment_type_size,payment_type_mode,payment_installments_count,payment_installments_min,payment_installments_max,payment_installments_mean,payment_installments_median,payment_installments_q25,payment_installments_q75,payment_installments_range,payment_value_count,payment_value_min,payment_value_max,payment_value_mean,payment_value_median,payment_value_q25,payment_value_q75,payment_value_range,product_id_count,seller_id_count,price_count,price_min,price_max,price_mean,price_median,price_q25,price_q75,price_range,freight_value_count,freight_value_min,freight_value_max,freight_value_mean,freight_value_median,freight_value_q25,freight_value_q75,freight_value_range,product_category_name_count,product_category_name_nunique,product_category_name_size,product_category_name_mode,product_name_lenght_count,product_name_lenght_min,product_name_lenght_max,product_name_lenght_mean,product_name_lenght_median,product_name_lenght_q25,product_name_lenght_q75,product_name_lenght_range,product_description_lenght_count,product_description_lenght_min,product_description_lenght_max,product_description_lenght_mean,product_description_lenght_median,product_description_lenght_q25,product_description_lenght_q75,product_description_lenght_range,product_photos_qty_count,product_photos_qty_min,product_photos_qty_max,product_photos_qty_mean,product_photos_qty_median,product_photos_qty_q25,product_photos_qty_q75,product_photos_qty_range,product_weight_g_count,product_weight_g_min,product_weight_g_max,product_weight_g_mean,product_weight_g_median,product_weight_g_q25,product_weight_g_q75,product_weight_g_range,product_length_cm_count,product_length_cm_min,product_length_cm_max,product_length_cm_mean,product_length_cm_median,product_length_cm_q25,product_length_cm_q75,product_length_cm_range,product_height_cm_count,product_height_cm_min,product_height_cm_max,product_height_cm_mean,product_height_cm_median,product_height_cm_q25,product_height_cm_q75,product_height_cm_range,product_width_cm_count,product_width_cm_min,product_width_cm_max,product_width_cm_mean,product_width_cm_median,product_width_cm_q25,product_width_cm_q75,product_width_cm_range,order_purchase_until_approved_in_seconds,order_purchase_until_approved_in_minutes,order_purchase_until_approved_in_hours,order_purchase_until_approved_in_days,order_purchase_timestamp_second,order_purchase_timestamp_minute,order_purchase_timestamp_hour,order_purchase_timestamp_day,order_purchase_timestamp_month,order_approved_at_second,order_approved_at_minute,order_approved_at_hour,order_approved_at_day,order_approved_at_month,delivered_in_days,estimated_delivery_in_days,time
1,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13,47813,barreiras,BA,-12.169860,-44.988369,1,1,1,belo horizonte,1,1,1,MG,1,852.256379,852.256379,852.256379,852.256379,852.256379,852.256379,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,boleto,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,141.46,141.46,141.46,141.46,141.46,141.46,0.0,1.0,1.0,1.0,118.7,118.7,118.7,118.7,118.7,118.7,0.0,1.0,22.76,22.76,22.76,22.76,22.76,22.76,0.0,1.0,1.0,1.0,perfumaria,1.0,29.0,29.0,29.0,29.0,29.0,29.0,0.0,1

## Data Dimensions

In [9]:
print('Number of Rows: {}'.format(df_train.shape[0]))
print('Number of Cols: {}'.format(df_train.shape[1]))

Number of Rows: 77853
Number of Cols: 147


In [10]:
print('Number of Rows: {}'.format(df_oot.shape[0]))
print('Number of Cols: {}'.format(df_oot.shape[1]))

Number of Rows: 18603
Number of Cols: 147


## Splitting into X and y

In [11]:
list_columns = [
    'order_purchase_timestamp', 'order_delivered_customer_date', 
    'order_estimated_delivery_date', 'delivered_in_days', 'estimated_delivery_in_days', 'time']

X_train = df_train.drop(columns=list_columns)
X_oot = df_oot.drop(columns=list_columns)

In [12]:
target = 'delivered_in_days'
list_columns = ['delivered_in_days', 'estimated_delivery_in_days']

y_train = df_train.loc[:, list_columns]
y_oot = df_oot.loc[:, list_columns]

# Machine Learning Modeling

## Training

In [13]:
target = 'delivered_in_days'

In [14]:
list_selected_features = [
    'distance_customer_seller_min',
    'product_height_cm_min',
    'distance_customer_seller_q25',
    'freight_value_min',
    'customer_lat',
    'order_approved_at_second',
    'customer_lng',
    'order_purchase_timestamp_month',
    'product_length_cm_min',
    'product_weight_g_min',
    'payment_value_q75',
    'payment_value_max',
    'payment_value_mean',
    'order_approved_at_month']

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'bagging_freq': 1,
    'random_state': random_state,
    'n_estimators': 832, 
    'learning_rate': 0.006365688565352602, 
    'num_leaves': 867, 
    'subsample': 0.6922606184521125, 
    'colsample_bytree': 0.4826899205085819, 
    'min_data_in_leaf': 52}

dict_cv_results = cross_validate(
    LGBMRegressor(**params), X=X_train[list_selected_features], y=y_train[target], 
    scoring={'r2': 'r2', 'mae': 'neg_mean_absolute_error', 'rmse': 'neg_root_mean_squared_error'}, 
    cv=3, n_jobs=-1)

get_cross_validate_metrics(dict_cv_results)


    R2 test mean: 0.3008610346273051; 
    R2 test std: 0.005168349210662973

    MAE test mean: -5.123517471052414; 
    MAE test std: 0.030781855582665323
    
    RMSE test mean: -8.347833620063584; 
    RMSE test std: 0.19346030702225256
    


In [15]:
lgbm = LGBMRegressor(**params)
lgbm.fit(X_train[list_selected_features], y_train[target])

y_train['pred'] = lgbm.predict(X_train[list_selected_features])

y_oot['pred'] = lgbm.predict(X_oot[list_selected_features])
get_metrics(y_oot[target], y_oot['pred'])

R2: 0.010447901186684594; RMSE: 5.776548950980762; MAE: 4.355468761199317; MAPE: 114987946674777.47


In [16]:
joblib.dump(lgbm, '../models/v5_machine_learning.joblib')

['../models/v5_machine_learning.joblib']

# API

In [125]:
import json
import joblib


def get_prediction(df):

    list_selected_features = [
        'distance_customer_seller_min',
        'product_height_cm_min',
        'distance_customer_seller_q25',
        'freight_value_min',
        'customer_lat',
        'order_approved_at_second',
        'customer_lng',
        'order_purchase_timestamp_month',
        'product_length_cm_min',
        'product_weight_g_min',
        'payment_value_q75',
        'payment_value_max',
        'payment_value_mean',
        'order_approved_at_month']
    
    model = joblib.load('../models/v5_machine_learning.joblib')

    df.loc[:, 'pred'] = model.predict(df[list_selected_features])

    return df


def get_date_prediction(df):

    df['pred_int'] = df['pred'].apply(np.round).astype(int)
    
    df['date_pred'] = df.apply(lambda i: i['order_purchase_timestamp'] + pd.DateOffset(day=i['pred_int']), axis=1)

    df['date_pred_max'] = df['date_pred'] + pd.DateOffset(days=4)

    df['date_pred_min'] = df['date_pred'] - pd.DateOffset(days=4)
    
    return df


def get_result(df):

    df = get_prediction(df)

    df = get_date_prediction(df)

    pred_max = df['date_pred_max'].dt.strftime('%Y-%m-%d')[0]
    pred_min = df['date_pred_min'].dt.strftime('%Y-%m-%d')[0]

    json_result = {'date_min': pred_min, 'date_max': pred_max}
    
    return json.dumps(json_result)

In [126]:
df = pd.read_json('../src/models/schema.json')

df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S')
df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d')

get_result(df)

'{"date_min": "2018-07-06", "date_max": "2018-07-14"}'